In [2]:
%%time

import os
import boto3
import sagemaker

from sagemaker import get_execution_role

region = boto3.Session().region_name

role = get_execution_role()

CPU times: user 688 ms, sys: 110 ms, total: 799 ms
Wall time: 1.22 s


Create S3 bucket

In [3]:
# This creates a default S3 bucket where we will upload our model.
bucket = sagemaker.Session().default_bucket()

In [4]:
bucket_path = "https://s3-{}.amazonaws.com/{}".format(region, bucket)

In [5]:
print(role)
print(region)
print(bucket)
print(bucket_path)

arn:aws:iam::208779919433:role/service-role/AmazonSageMakerServiceCatalogProductsExecutionRole
ap-south-1
sagemaker-ap-south-1-208779919433
https://s3-ap-south-1.amazonaws.com/sagemaker-ap-south-1-208779919433


In [6]:
!conda install -y -c conda-forge xgboost==1.5.0

Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - xgboost==1.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _openmp_mutex-5.1          |            1_gnu          21 KB
    _py-xgboost-mutex-2.0      |            cpu_0           8 KB  conda-forge
    libgomp-11.2.0             |       h1234567_1         474 KB
    libxgboost-1.5.0           |       h295c915_1         2.0 MB
    openssl-1.1.1q             |       h7f8727e_0         2.5 MB
    py-xgboost-1.5.0           |   py37h06a4308_1         162 KB
    xgboost-1.5.0              |   py37h06a4308_1          25 KB
    ------------------------------------------------------------
                                           Total:         5.2 MB

The following NEW packages will be INSTALLED:

  _openmp_mutex      pkgs/main/linux-64::_openmp_mutex-5.1-1_gnu
  _py-xgboost-

In [7]:
model_file_name = "DEMO-local-xgboost-model-fraud-detection"

Load the pre-trained model and test it before deployment

In [8]:
import joblib
import xgboost

mymodel = joblib.load(model_file_name)

In [9]:
#import json
import numpy as np


file_name = (
    "test_point_fraud.csv"  # customize to your test file, will be 'mnist.single.test' if use data above
)

with open(file_name, "r") as f:
    mypayload = np.loadtxt(f, delimiter=",")
    
print(mypayload)

[[ 5.00000e+00  2.60000e+01  0.00000e+00  1.00000e+00  2.00000e+03
   1.13702e+03  0.00000e+00  0.00000e+00  6.00000e+00  4.00000e+00
   1.60000e+01  1.00000e+00  3.15000e+04  0.00000e+00  2.00000e+00
   1.00000e+00  2.00000e+00  0.00000e+00  6.00000e+00  6.00000e+00
   2.97000e+02  2.20000e+01  1.00000e+00  1.00000e+00  1.00000e+00
   3.00000e+00  0.00000e+00  8.81100e+04  1.60200e+04  1.60200e+04
   5.60700e+04  1.00000e+00  5.00000e+00  2.00300e+03]
 [ 1.60000e+02  3.30000e+01  0.00000e+00  2.00000e+00  1.00000e+03
   1.42278e+03  0.00000e+00  0.00000e+00  2.00000e+00  3.00000e+00
   8.00000e+00  0.00000e+00  6.16000e+04  0.00000e+00  0.00000e+00
   0.00000e+00  2.00000e+00  0.00000e+00  0.00000e+00  5.00000e+00
   1.11000e+02  1.70000e+01  3.00000e+00  0.00000e+00  2.00000e+00
   3.00000e+00  1.00000e+00  5.28000e+04  5.28000e+03  5.28000e+03
   4.22400e+04  9.00000e+00  2.90000e+01  2.00600e+03]
 [ 3.85000e+02  5.10000e+01  1.00000e+00  1.00000e+00  1.00000e+03
   9.76370e+02  0.0

In [10]:
mymodel.predict(mypayload)

array([0, 0, 0, 0, 0], dtype=int32)

In [11]:
#### This step Booster.save_model was needed before creating a tar.gz . Otherwise I faced issues with prediction on deployment.

mymodel._Booster.save_model(model_file_name)

In [12]:
!tar czvf model.tar.gz $model_file_name

DEMO-local-xgboost-model-fraud-detection


Upload the pre-trained model to S3:

In [13]:
#### prefix in S3
prefix = "Sagemaker/DEMO-XGBoost-Auto-Insurance"

fObj = open("model.tar.gz", "rb")
key = os.path.join(prefix, model_file_name, "model.tar.gz")
print(key)
boto3.Session().resource("s3").Bucket(bucket).Object(key).upload_fileobj(fObj)

Sagemaker/DEMO-XGBoost-Auto-Insurance/DEMO-local-xgboost-model-fraud-detection/model.tar.gz


Set up hosting for the model:

Import model into hosting

This involves creating a SageMaker model from the model file previously uploaded to S3.

Create a Sagemaker model:

In [14]:
from sagemaker.amazon.amazon_estimator import get_image_uri

#### Get the built-in xgboost container image in Sagemaker to host our model
container = get_image_uri(boto3.Session().region_name, "xgboost", "1.5-1")

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [15]:
%%time
from time import gmtime, strftime

model_name = model_file_name + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

model_url = "https://s3-{}.amazonaws.com/{}/{}".format(region, bucket, key)

sm_client = boto3.client("sagemaker")

print(model_url)

primary_container = {
    "Image": container,
    "ModelDataUrl": model_url,
}

create_model_response2 = sm_client.create_model(
    ModelName=model_name, ExecutionRoleArn=role, PrimaryContainer=primary_container
)

print(create_model_response2["ModelArn"])

https://s3-ap-south-1.amazonaws.com/sagemaker-ap-south-1-208779919433/Sagemaker/DEMO-XGBoost-Auto-Insurance/DEMO-local-xgboost-model-fraud-detection/model.tar.gz
arn:aws:sagemaker:ap-south-1:208779919433:model/demo-local-xgboost-model-fraud-detection2022-07-26-08-38-09
CPU times: user 59.9 ms, sys: 16.8 ms, total: 76.7 ms
Wall time: 450 ms


Create endpoint configuration:

Create an endpoint configuration, that describes the distribution of traffic across the models, whether split, shadowed, or sampled in some way. In addition, the endpoint configuration describes the instance type required for model deployment.

In [16]:
from time import gmtime, strftime

endpoint_config_name = "DEMO-XGBoostEndpointConfig-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

print(endpoint_config_name)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": "ml.m4.xlarge",
            "InitialInstanceCount": 1,
            "InitialVariantWeight": 1,
            "ModelName": model_name,
            "VariantName": "AllTraffic",
        }
    ],
)

print("Endpoint Config Arn: " + create_endpoint_config_response["EndpointConfigArn"])

DEMO-XGBoostEndpointConfig-2022-07-26-08-38-21
Endpoint Config Arn: arn:aws:sagemaker:ap-south-1:208779919433:endpoint-config/demo-xgboostendpointconfig-2022-07-26-08-38-21


Create endpoint:

Lastly, you create the endpoint that serves up the model, through specifying the name and configuration defined above. The end result is an endpoint that can be validated and incorporated into production applications. This takes 4-5 minutes to complete.

In [17]:
%%time
import time

endpoint_name = "DEMO-XGBoostEndpoint-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_name)
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)
print(create_endpoint_response["EndpointArn"])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

DEMO-XGBoostEndpoint-2022-07-26-08-38-30
arn:aws:sagemaker:ap-south-1:208779919433:endpoint/demo-xgboostendpoint-2022-07-26-08-38-30
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagemaker:ap-south-1:208779919433:endpoint/demo-xgboostendpoint-2022-07-26-08-38-30
Status: InService
CPU times: user 59.7 ms, sys: 7.56 ms, total: 67.3 ms
Wall time: 3min


Validate the model for use:

Now you can obtain the endpoint from the client library using the result from previous operations and generate classifications from the model using that endpoint.

In [18]:
runtime_client = boto3.client("runtime.sagemaker")

Lets generate the prediction. We'll pick csv data from the test data file

In [19]:
%%time
import json


file_name = (
    "test_point_fraud.csv"  # customize to your test file, will be 'mnist.single.test' if use data above
)

with open(file_name, "r") as f:
    payload = f.read().strip()
    
    
print("Payload :\n")

print(payload)
print()

response = runtime_client.invoke_endpoint(
    EndpointName=endpoint_name, ContentType="text/csv", Body=payload
)

##print(response)

print("Results :\n")
print()

result = response["Body"].read().decode("ascii")

# Unpack response
print("\nPredicted Class Probabilities: {}.".format(result))

Payload :

5.000000000000000000e+00,2.600000000000000000e+01,0.000000000000000000e+00,1.000000000000000000e+00,2.000000000000000000e+03,1.137019999999999982e+03,0.000000000000000000e+00,0.000000000000000000e+00,6.000000000000000000e+00,4.000000000000000000e+00,1.600000000000000000e+01,1.000000000000000000e+00,3.150000000000000000e+04,0.000000000000000000e+00,2.000000000000000000e+00,1.000000000000000000e+00,2.000000000000000000e+00,0.000000000000000000e+00,6.000000000000000000e+00,6.000000000000000000e+00,2.970000000000000000e+02,2.200000000000000000e+01,1.000000000000000000e+00,1.000000000000000000e+00,1.000000000000000000e+00,3.000000000000000000e+00,0.000000000000000000e+00,8.811000000000000000e+04,1.602000000000000000e+04,1.602000000000000000e+04,5.607000000000000000e+04,1.000000000000000000e+00,5.000000000000000000e+00,2.003000000000000000e+03
1.600000000000000000e+02,3.300000000000000000e+01,0.000000000000000000e+00,2.000000000000000000e+00,1.000000000000000000e+03,1.422779999999

Delete the Endpoint:

In [ ]:
sm_client.delete_endpoint(EndpointName=endpoint_name)

# Register and Deploy Models with Model Registry:

###### 1.Create a Model Group:

A model group contains a group of versioned models. Create a model group by using either the AWS SDK for Python (Boto3) or Amazon SageMaker Studio.

To create a model group by using Boto3, call the create_model_package_group method and specify a name and description as parameters. The following example shows how to create a model group. The response from the create_model_package_group call is the Amazon Resource Name (ARN) of the new model package group.

First, import the required packages and set up the SageMaker Boto3 client.

In [20]:
import time
import os
from sagemaker import get_execution_role, session
import boto3

region = boto3.Session().region_name

role = get_execution_role()

sm_client = boto3.client('sagemaker', region_name=region)

Now create the model group.

In [21]:
import time
model_package_group_name = "DEMO-local-xgboost-model-fraud-detection" + str(round(time.time()))
model_package_group_input_dict = {
 "ModelPackageGroupName" : model_package_group_name,
 "ModelPackageGroupDescription" : "Sample model package group"
}

create_model_package_group_response = sm_client.create_model_package_group(**model_package_group_input_dict)
print('ModelPackageGroup Arn : {}'.format(create_model_package_group_response['ModelPackageGroupArn']))

ModelPackageGroup Arn : arn:aws:sagemaker:ap-south-1:208779919433:model-package-group/demo-local-xgboost-model-fraud-detection1658824993


###### 2.Register a Model Version:

You can register an Amazon SageMaker model by creating a model version that specifies the model group to which it belongs. A model version must include both the model artifacts (the trained weights of a model) and the inference code for the model.

To register a model version by using Boto3, call the create_model_package method.

First, you set up the parameter dictionary to pass to the create_model_package method.

In [22]:
# Specify the model source

model_url = "https://s3-{}.amazonaws.com/{}/{}".format(region, bucket, key)

modelpackage_inference_specification =  {
    "InferenceSpecification": {
      "Containers": [
         {
            "Image": container,
	    "ModelDataUrl": model_url
         }
      ],
      "SupportedContentTypes": [ "text/csv" ],
      "SupportedResponseMIMETypes": [ "text/csv" ],
   }
 }

# Alternatively, you can specify the model source like this:
# modelpackage_inference_specification["InferenceSpecification"]["Containers"][0]["ModelDataUrl"]=model_url

create_model_package_input_dict = {
    "ModelPackageGroupName" : model_package_group_name,
    "ModelPackageDescription" : "Model to detect wheather the claim is fraud or non-fraud",
    "ModelApprovalStatus" : "PendingManualApproval"
}
create_model_package_input_dict.update(modelpackage_inference_specification)

Then you call the create_model_package method, passing in the parameter dictionary that you just set up.

In [23]:
create_model_package_response = sm_client.create_model_package(**create_model_package_input_dict)
model_package_arn = create_model_package_response["ModelPackageArn"]
print('ModelPackage Version ARN : {}'.format(model_package_arn))

ModelPackage Version ARN : arn:aws:sagemaker:ap-south-1:208779919433:model-package/demo-local-xgboost-model-fraud-detection1658824993/1


###### 3.View Model Groups and Versions:

Model groups and versions help you organize your models. You can view a list of the model versions in a model group.

You can view all of the model versions that are associated with a model group. If a model group represents all models that you train to address a specific ML problem, you can view all of those related models.

To view model versions associated with a model group by using Boto3, call the list_model_packages method, and pass the name of the model group as the value of the ModelPackageGroupName parameter.

In [24]:
sm_client.list_model_packages(ModelPackageGroupName=model_package_group_name)

{'ModelPackageSummaryList': [{'ModelPackageGroupName': 'DEMO-local-xgboost-model-fraud-detection1658824993',
   'ModelPackageVersion': 1,
   'ModelPackageArn': 'arn:aws:sagemaker:ap-south-1:208779919433:model-package/demo-local-xgboost-model-fraud-detection1658824993/1',
   'ModelPackageDescription': 'Model to detect wheather the claim is fraud or non-fraud',
   'CreationTime': datetime.datetime(2022, 7, 26, 8, 44, 29, 679000, tzinfo=tzlocal()),
   'ModelPackageStatus': 'Completed',
   'ModelApprovalStatus': 'PendingManualApproval'}],
 'ResponseMetadata': {'RequestId': '2d8c3a11-b659-4f28-b20d-a92f0dadc20c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '2d8c3a11-b659-4f28-b20d-a92f0dadc20c',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '457',
   'date': 'Tue, 26 Jul 2022 08:45:37 GMT'},
  'RetryAttempts': 0}}

###### 4.View the Details of a Model Version:

You can view details of a specific model version by using either the AWS SDK for Python (Boto3) or by using Amazon SageMaker Studio.

To view the details of a model version by using Boto3, complete the following steps.

1.Call the list_model_packages method to view the model versions in a model group.

In [29]:
sm_client.list_model_packages(ModelPackageGroupName="ModelGroup1")

{'ModelPackageSummaryList': [],
 'ResponseMetadata': {'RequestId': '1edeb775-df06-4cc6-9105-c9e93cc93183',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '1edeb775-df06-4cc6-9105-c9e93cc93183',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '30',
   'date': 'Tue, 26 Jul 2022 08:55:40 GMT'},
  'RetryAttempts': 0}}

The response is a list of model package summaries. You can get the Amazon Resource Name (ARN) of the model versions from this list.

2.Call describe_model_package to see the details of the model version. You pass in the ARN of a model version that you got in the output of the call to list_model_packages.

In [26]:
sm_client.describe_model_package(ModelPackageName="arn:aws:sagemaker:ap-south-1:208779919433:model-package/demo-local-xgboost-model-fraud-detection1658824993/1")

{'ModelPackageGroupName': 'DEMO-local-xgboost-model-fraud-detection1658824993',
 'ModelPackageVersion': 1,
 'ModelPackageArn': 'arn:aws:sagemaker:ap-south-1:208779919433:model-package/demo-local-xgboost-model-fraud-detection1658824993/1',
 'ModelPackageDescription': 'Model to detect wheather the claim is fraud or non-fraud',
 'CreationTime': datetime.datetime(2022, 7, 26, 8, 44, 29, 679000, tzinfo=tzlocal()),
 'InferenceSpecification': {'Containers': [{'Image': '720646828776.dkr.ecr.ap-south-1.amazonaws.com/sagemaker-xgboost:1.5-1',
    'ImageDigest': 'sha256:900372db4cdb1f34e9eae2de344350b53b4d7c141e883a1e197fd3576c73d2a6',
    'ModelDataUrl': 'https://s3-ap-south-1.amazonaws.com/sagemaker-ap-south-1-208779919433/Sagemaker/DEMO-XGBoost-Auto-Insurance/DEMO-local-xgboost-model-fraud-detection/model.tar.gz'}],
  'SupportedContentTypes': ['text/csv'],
  'SupportedResponseMIMETypes': ['text/csv']},
 'ModelPackageStatus': 'Completed',
 'ModelPackageStatusDetails': {'ValidationStatuses': [],

The output of this call is a JSON object with the model version details.

###### 5.Update the Approval Status of a Model:

After you create a model version, you typically want to evaluate its performance before you deploy it to a production endpoint. If it performs to your requirements, you can update the approval status of the model version to Approved. Setting the status to Approved can initiate CI/CD deployment for the model. If the model version does not perform to your requirements, you can update the approval status to Rejected.

When you use one of the SageMaker provided project templates and the approval status of a model version changes, the following action occurs. Only valid transitions are shown.

PendingManualApproval to Approved – initiates CI/CD deployment for the approved model version

PendingManualApproval to Rejected – No action

Rejected to Approved – initiates CI/CD deployment for the approved model version

Approved to Rejected – initiates CI/CD to deploy the latest model version with an Approved status

When you created the model version in Register a Model Version, you set the ModelApprovalStatus to PendingManualApproval. You update the approval status for the model by calling update_model_package. Note that you can automate this process by writing code that, for example, sets the approval status of a model depending on the result of an evaluation of some measure of the model's performance. You can also create a step in a pipeline that automatically deploys a new model version when it is approved. The following code snippet shows how to manually change the approval status to Approved.

In [27]:
model_package_update_input_dict = {
    "ModelPackageArn" : model_package_arn,
    "ModelApprovalStatus" : "Approved"
}
model_package_update_response = sm_client.update_model_package(**model_package_update_input_dict)

In [28]:
sm_client.list_model_packages(ModelPackageGroupName=model_package_group_name)

{'ModelPackageSummaryList': [{'ModelPackageGroupName': 'DEMO-local-xgboost-model-fraud-detection1658824993',
   'ModelPackageVersion': 1,
   'ModelPackageArn': 'arn:aws:sagemaker:ap-south-1:208779919433:model-package/demo-local-xgboost-model-fraud-detection1658824993/1',
   'ModelPackageDescription': 'Model to detect wheather the claim is fraud or non-fraud',
   'CreationTime': datetime.datetime(2022, 7, 26, 8, 44, 29, 679000, tzinfo=tzlocal()),
   'ModelPackageStatus': 'Completed',
   'ModelApprovalStatus': 'Approved'}],
 'ResponseMetadata': {'RequestId': 'd512c257-8012-4037-9b86-e49a8adeafdc',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd512c257-8012-4037-9b86-e49a8adeafdc',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '444',
   'date': 'Tue, 26 Jul 2022 08:48:52 GMT'},
  'RetryAttempts': 0}}

###### 6.Deploy a Model from the Registry:


After you register a model version and approve it for deployment, deploy it to a SageMaker endpoint for real-time inference.

When you create an MLOps project and choose an MLOps project template that includes model deployment, approved model versions in the model registry are automatically deployed to production.

Deploy a Model from the Registry (Boto3)

To deploy a model version using the AWS SDK for Python (Boto3), complete the following steps:

1.Create a model object from the model version by calling the create_model method. Pass the Amazon Resource Name (ARN) of the model version as part of the Containers for the model object.

The following code snippet assumes you have already created the SageMaker Boto3 client sm_client, and that you have already created a model version with an ARN that you have stored in a variable named model_version_arn.

model_name = 'DEMO-modelregistry-model-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("Model name : {}".format(model_name))
container_list = [{'ModelPackageName': model_version_arn}]

create_model_response = sm_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    Containers = container_list
)
print("Model arn : {}".format(create_model_response["ModelArn"]))

2.Create an endpoint configuration by calling create_endpoint_config. The endpoint configuration specifies the number and type of Amazon EC2 instances to use for the endpoint.

endpoint_config_name = 'DEMO-modelregistry-EndpointConfig-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_config_name)
create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType':'ml.m4.xlarge',
        'InitialVariantWeight':1,
        'InitialInstanceCount':1,
        'ModelName':model_name,
        'VariantName':'AllTraffic'}])

3.Create the endpoint by calling create_endpoint.

endpoint_name = 'DEMO-modelregistry-endpoint-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointName={}".format(endpoint_name))

create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print(create_endpoint_response['EndpointArn'])

7.View the Deployment History of a Model:

View the deployments for a model version Amazon SageMaker Studio by opening the tab for that model version.
View the deployment history for a model version

1.Sign in to Studio. For more information, see Onboard to Amazon SageMaker Domain.

2.In the left navigation pane, choose the SageMaker Resources icon (  ).

3.Choose Model registry.

4.From the model groups list, choose the model group you want to view.

5.A new tab appears with a list of the model versions in the model group.

6.In the list of model versions, double-click the model version for which you want to view details.

7.On the model version tab that opens, choose Activity. Deployments for the model version appear as events in the activity list with an Event type of ModelDeployment.

8.Delete a Model Group:

Delete a Model Package Group (Amazon SageMaker Studio)
To delete a model group in Amazon SageMaker Studio, complete the following steps.

1.Sign in to Amazon SageMaker Studio. For more information, see Onboard to Amazon SageMaker Domain.

2.In the left navigation pane, choose the SageMaker Resources icon (  ).

3.Choose Model registry in the dropdown menu at the top of the SageMaker resources panel.
A list of your model groups appears.

4.From the model groups list, double-click the model group you want to delete.
The model details tab opens to the right.

5.In the Actions dropdown menu in the top right corner of the model details tab, choose Delete.

6.In the confirmation dialog box, choose Delete.

9.Delete a Model Version:

Delete a Model Version (Amazon SageMaker Studio):
1.To delete a model version in Amazon SageMaker Studio, complete the following steps.

2.Sign in to Amazon SageMaker Studio. For more information, see Onboard to Amazon SageMaker Domain.

3.In the left navigation pane, choose the SageMaker Resources icon (  ).

4.Choose Model registry in the dropdown menu at the top of the SageMaker resources panel.

A list of your model groups appears.

5.From the model groups list, double-click the model group of the model version you want to delete.

The model details tab opens to the right.

6.From the list of model versions in the model details tab, double-click the model version you want to delete.
Choose the Settings tab.

7.Choose Delete Version in the Settings tab.
8.In the confirmation dialog box, choose Delete.